In [3]:
import speech_recognition as sr
import subprocess
from gtts import gTTS
from bs4 import BeautifulSoup
import requests
import spacy
from json import load
from numpy import array, dot, asarray, exp
from nltk import word_tokenize
from nltk.stem.lancaster import LancasterStemmer


stemmer = LancasterStemmer()


classes = ['remainder', 'question', 'status', 'collage']
words = ['had', "'", 'in', 'for', 'me', 'dat', 'i', 'stat', 'ar', 'which', 'ev', 'holiday', 'that', 'grad', 'it', 'we', 'plac', 'al', 'tel', 'string', 'syllab', 'a', 'is', 'when', 'my', 'of', 'hap', 'subject', 'cgpa', 'cur', 'weath', "'s", 'what', 'int', 'on', 'visit', 'company', 'sem', 'do', 'formul', "'int", 'thi', 's', 'today', 'the', 'remaind', 'hav', 'mark', 'wil', '’', 'set', 'did']


nlp = spacy.load("en_core_web_lg")

headers = {
	'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
	}
ERROR_THRESHOLD = 0.2

synapse_file = 'synapses.json' 
with open(synapse_file) as data_file: 
    synapse = load(data_file) 
    synapse_0 = asarray(synapse['synapse0']) 
    synapse_1 = asarray(synapse['synapse1'])

r = sr.Recognizer()
r.energy_threshold = 10000

words_ = ['me', "tell"]

In [4]:

def speechToText(do_say=False):
    with sr.Microphone() as source:
        print('Recognizing..')
        audio = r.listen(source)
        print("Done")

    try:
        text = r.recognize_google(audio)
        if do_say:
            say(text)
        return text

    except Exception as e:
        print(e)

def activate(phrase='hey dude'):
    try:
        with sr.Microphone() as source:
            r.adjust_for_ambient_noise(source)
            audio = r.listen(source)
            transcript = r.recognize_google(audio)
            if transcript.lower() == phrase:
                return speechToText(do_say=False)
            else:
                return False
    except:
        print("error")


def is_similar(base_word, tokens):
    base_word = nlp(base_word)
    tokens = nlp(tokens)
    similar_word = {}

    for token in tokens:
        similar = base_word.similarity(token)
        boolean = False
        if similar > 0.5: boolean = True
        else: boolean = False
        similar_word[token] = [similar, boolean]
    return similar_word

def weather(city='gadag'):
    city = city.replace(" ", "+")
    res = requests.get(
        f'https://www.google.com/search?q={city}&oq={city}&aqs=chrome.0.35i39l2j0l4j46j69i60.6128j1j7&sourceid=chrome&ie=UTF-8',
        headers=headers
        )
    print("Searching...\n")
    soup = BeautifulSoup(res.text, 'html.parser')
    location = soup.select('#wob_loc')[0].getText().strip()
    time = soup.select('#wob_dts')[0].getText().strip()
    info = soup.select('#wob_dc')[0].getText().strip()
    weather = soup.select('#wob_tm')[0].getText().strip()

    print(location)
    print(time)
    print(info)
    print(weather+"°C")
    return f'weather at {location} at time {time} is {weather}°C'

 
def weather1(city):
    city = city.replace(" ", "+")
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
 
    res = requests.get(
        f'https://www.google.com/search?q={city}&oq={city}&aqs=chrome.0.35i39l2j0l4j46j69i60.6128j1j7&sourceid=chrome&ie=UTF-8', headers=headers)
    print("Searching...\n")
    
    soup = BeautifulSoup(res.text, 'html.parser')
    print(soup.prettify())
    return
    location = soup.select('#wob_loc')[0].getText().strip()
    time = soup.select('#wob_dts')[0].getText().strip()
    info = soup.select('#wob_dc')[0].getText().strip()
    weather = soup.select('#wob_tm')[0].getText().strip()
    print(location)
    print(time)
    print(info)
    print(weather+"°C")

def sigmoid(x):
    output = 1/(1+exp(-x))
    return output


def sigmoid_output_to_derivative(output):
    return output*(1-output)
 
def clean_up_sentence(sentence):
    sentence_words = word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words


def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(array(bag))

def think(sentence, show_details=False):
    x = bow(sentence.lower(), words, show_details)
    if show_details:
        print ("sentence:", sentence, "\n bow:", x)
    l0 = x
    l1 = sigmoid(dot(l0, synapse_0))
    l2 = sigmoid(dot(l1, synapse_1))
    return l2



def classify(sentence, show_details=False):
    results = think(sentence, show_details)

    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD ] 
    results.sort(key=lambda x: x[1], reverse=True) 
    return_results =[[classes[r[0]],r[1]] for r in results]

    return return_results

def preprocess(text):
    doc = nlp(text)
    filtered_tokens = list()
    for token in doc:
        if  token.is_stop or  token.is_punct :
            continue
        filtered_tokens.append(token.lemma_)
    return " ".join(filtered_tokens)

def say(text, tld='co.in', lang='en' ,slow=False):
    tts = gTTS(text, tld, lang ,slow=False)
    tts.save('hello.mp3')
    subprocess.call(['afplay', 'hello.mp3'])

In [57]:
import re as reg
text = 'tell me my 4th sem ESA of UE21CS303 grades'

def college(text):
    if "grade" in text or "marks" in text:
        if "ISA" in text: 
            ISA(text)
            return
        ESA(text)

def ESA(text):
    sem_pattern_th = '(\drd|\dst|\dnd|\dth) (sem)'
    sem_pattern = '(\d)'
    res = reg.search(sem_pattern_th, text)
    if res == None:
        res = reg.search(sem_pattern, text)
        if res == None: return
    
    # (int(res.start()) < 0 and int(res.start()) > 8)


    sem_grades = {
        "3" : "Your Grade As of 3rd is 8.04 CGPA",
        "4" : "Your Grade As of 4th is 8.19 CGPA",
        "5" : "Your Grade As of 5th is 7.77 CGPA",
        "6" : "Your current Grade is 7.89 CGPA as of 6th sem"
    }

    say(sem_grades[text[res.start()]])

def ISA(text):
    sem_pattern_th = '(\drd|\dst|\dnd|\dth) (sem)'
    sem_pattern = '(\d)'
    res = reg.search(sem_pattern_th, text)
    if res == None:
        res = reg.search(sem_pattern, text)
        if res == None: return

    sem = res.start()
    course_pattern = 'UE\d\dCS\d\d\d'
    res = reg.search(course_pattern, text)
    

    sem_grades = {
        "3" : {
            "UE21CS303":"3rd sem"
        },
        "4" : {
            "UE21CS303":"4th"
        },
        "5" : {
            "UE21CS303":"5th"
        },
        "6" : {
            "UE21CS303":"6th"
        },
    }

    say(sem_grades[text[sem]][text[res.start():res.end()]])


college(text)


In [84]:
def college(text):
    if "grade" in text:
        say("your current Grade is 7.8 CGPA",  slow=False)

def status(text):
    if "placement" in text:
        say("currently Accenture is hiring" , slow=False)

def set_remaind(text):
    if "remaind" in text:
        say("remainder is set" , slow=False)

def question(text):
    print(text)
    if "weather" in text:
        say("weather is clear" ,slow=False)
    

In [60]:
def name():
    say("My name is Bubbles, I'm at your Service",  slow=False)

def skills(text):
    if "name" in text:
        name()
        
    resp = classify(text)
    
    if len(resp) > 0:
        if resp[0][1] > 0.70:
            if resp[0][0] == 'collage':
                college(text)
            if resp[0][0] == 'status':
                status(text)
            if resp[0][0] == 'remainder':
                set_remaind(text)
            if resp[0][0] == 'question':
                question(text)
    print(resp)


def main():
    text = speechToText()
    # text = "tell me my grades of 5th sem"
    # text = "whats your name"
    # text = activate()
    text = preprocess(text)
    print(text)

    if skills(text):
        print("yes")


main()

Recognizing..
Done
tell fifth sem grade
[['collage', 0.9957921028180631]]
